# Filter single node differences
We have tons of single node differences - how can we filter them?

In [1]:
import json
import difflib
from IPython.core.display import display, HTML
from diff_match_patch import diff_match_patch

from colorama import Fore
import ipysheet

import re

In [2]:
SINGLE_NODE_DIFF_PATH = "/projects/bdata/datasets/kaggle-competitions/processed/matches.jsonl"

In [3]:
!head $SINGLE_NODE_DIFF_PATH -n 5

[{"slug": "allen21huang", "version_id": "23387521", "source": "# check and change our directory\nos.chdir('/kaggle/input/natural-images/data/natural_images')\nprint(os.listdir())"}, {"slug": "allen21huang", "version_id": "23383256", "source": "os.chdir('/kaggle/input')\nprint(os.listdir())"}]
[{"slug": "allen21huang", "version_id": "23387521", "source": "# check and change our directory\nos.chdir('/kaggle/input/natural-images/data/natural_images')\nprint(os.listdir())"}, {"slug": "allen21huang", "version_id": "23383256", "source": "os.chdir('/kaggle/input')\nprint(os.listdir())"}]
[{"slug": "allen21huang", "version_id": "23387521", "source": "# check and change our directory\nos.chdir('/kaggle/input/natural-images/data/natural_images')\nprint(os.listdir())"}, {"slug": "allen21huang", "version_id": "23383639", "source": "os.chdir('/kaggle/input')\nprint(os.listdir())"}]
[{"slug": "allen21huang", "version_id": "23387521", "source": "# check and change our directory\nos.chdir('/kaggle/inp

In [4]:
!wc -l $SINGLE_NODE_DIFF_PATH

23610029 /projects/bdata/datasets/kaggle-competitions/processed/matches.jsonl


In [5]:
single_node_samples = !shuf -n 10000 $SINGLE_NODE_DIFF_PATH
single_node_samples = [json.loads(x) for x in single_node_samples]

In [6]:
from io import StringIO
import tokenize
import token
import sys

def remove_comments(src):
    """
    This reads tokens using tokenize.generate_tokens and recombines them
    using tokenize.untokenize, and skipping comment/docstring tokens in between
    """
    f = StringIO(src)
    class SkipException(Exception): pass
    processed_tokens = []
    last_token = None
    # go thru all the tokens and try to skip comments and docstrings
    for tok in tokenize.generate_tokens(f.readline):
        t_type, t_string, t_srow_scol, t_erow_ecol, t_line = tok

        try:
            if t_type == tokenize.COMMENT:
                raise SkipException()

            elif t_type == tokenize.STRING:

                if last_token is None or last_token[0] in [tokenize.INDENT]:
                    # FIXEME: this may remove valid strings too?
                    #raise SkipException()
                    pass

        except SkipException:
            pass
        else:
            processed_tokens.append(tok)

        last_token = tok

    return tokenize.untokenize(processed_tokens)


def remove_comments_from_src(src):
    """
    This reads tokens using tokenize.generate_tokens and recombines them
    using tokenize.untokenize, and skipping comment/docstring tokens in between
    """
    try:
        f = StringIO(src)
        class SkipException(Exception): pass
        processed_tokens = []
        last_token = None
        # go thru all the tokens and try to skip comments and docstrings
        for tok in tokenize.generate_tokens(f.readline):
            t_type, t_string, t_srow_scol, t_erow_ecol, t_line = tok

            try:
                if t_type == tokenize.COMMENT:
                    raise SkipException()

                elif t_type == tokenize.STRING:

                    if last_token is None or last_token[0] in [tokenize.INDENT]:
                        # FIXEME: this may remove valid strings too?
                        #raise SkipException()
                        pass

            except SkipException:
                pass
            else:
                processed_tokens.append(tok)

            last_token = tok

        return tokenize.untokenize(processed_tokens)
    #I belive this happens with invalid python
    except (ValueError, tokenize.TokenError, IndentationError) as e:
        return src


def remove_comments_and_docstrings(src):
    
    source = StringIO(src)
    mod = StringIO()
    
    prev_toktype = token.INDENT
    first_line = None
    last_lineno = -1
    last_col = 0

    tokgen = tokenize.generate_tokens(source.readline)
    for toktype, ttext, (slineno, scol), (elineno, ecol), ltext in tokgen:
        if 0:   # Change to if 1 to see the tokens fly by.
            print("%10s %-14s %-20r %r" % (
                tokenize.tok_name.get(toktype, toktype),
                "%d.%d-%d.%d" % (slineno, scol, elineno, ecol),
                ttext, ltext
                ))
        if slineno > last_lineno:
            last_col = 0
        if scol > last_col:
            mod.write(" " * (scol - last_col))
#         if toktype == token.STRING and prev_toktype == token.INDENT:
#             # Docstring
#             continue
#         elif toktype == tokenize.COMMENT:
#             # Comment
#             continue
        elif not(toktype == tokenize.COMMENT) and not(toktype == token.STRING and prev_toktype == token.INDENT):
            mod.write(ttext)
        prev_toktype = toktype
        last_col = ecol
        last_lineno = elineno
    return mod.getvalue()

In [7]:
test = """
#train_images = train_dogs[:2000] + train_cats[:2000]
"""

In [8]:
print(remove_comments(test))

In [9]:
print(remove_comments_and_docstrings(test))

The following diffs use were generated using:
`python tree_matches --length_threshold 3 --ignore_function_args --remove_exact_duplicates --ignore_strings --sequential_matches`

In [10]:
examples = !cat /homes/gws/mikeam/RobustDataScience/matches.jsonl
examples = [json.loads(x) for x in examples]

In [11]:
# def html_diff(a,b):
#     differ = difflib.HtmlDiff(wrapcolumn = 80)
#     return differ.make_table(a.split("\n"),b.split("\n"), context = True)


def pprint_code_diff(a,b):
    dmp = diff_match_patch()
    diffs = dmp.diff_main(a,b)
    dmp.diff_cleanupSemantic(diffs)
    display(HTML(dmp.diff_prettyHtml(diffs)))
    
def compare_diff(examples,index):
    orig = examples[index][0]
    nn = examples[index][1]
    pprint_code_diff(orig["source"],nn["source"])

In [12]:
html_break = '<br><span style="color:red">------------------------------</span>'
for i in range(150,170):
    compare_diff(examples,i)
    display(HTML(html_break))

# What about the 'Kaggle Diffs'?

In [13]:
diff_examples = !shuf -n 10000 /homes/gws/mikeam/RobustDataScience/diffs_new.jsonl
diff_examples = [json.loads(x) for x in diff_examples]

In [14]:
def red(text):
    return (Fore.RED + text + Fore.RESET)

def green(text):
    return (Fore.GREEN + text + Fore.RESET)
    
def display_kaggle_diff(diff,return_string=False):
    diff_disp = get_diff_visual(diff)
    if return_string:
        return diff_disp
    else:
        print(diff_disp)
        
def get_diff_visual(diff):
    
    string_lines = []
    
    string_lines.append(Fore.BLUE + diff["original_path"])
    string_lines.append("-"*100 + Fore.RESET)
    
    for line in diff["cell_diff"].split("\n"):
        if len(line) == 0:
            string_lines.append("")
            continue
        if line[0] == "+":
            string_lines.append(green(line))
        elif line[0] == "-":
            string_lines.append(red(line))
        else:
            string_lines.append(line)
    return "\n".join(string_lines)


for diff in diff_examples[:40]:
    display_kaggle_diff(diff)

data/processed/competitions/imet-2019-fgvc6/mathormad/14614696.json
----------------------------------------------------------------------------------------------------

-# train all layers
+                  
 for layer in model.layers:
     layer.trainable = True
 
 callbacks_list = [checkpoint, csv_logger, reduceLROnPlat]
 model.compile(loss='binary_crossentropy',
-            # loss=focal_loss,
+                              
             optimizer=Adam(lr=1e-4))
-            # optimizer=AdamAccumulate(lr=1e-4, accum_iters=2))
+                                                               
 
 model.fit_generator(
     train_mixup,
     steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
     validation_data=validation_generator,
     validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
     epochs=epochs,
     verbose=1,
-    max_queue_size=16, workers=WORKERS, use_multiprocessing=True)
+    max_queue_size=16, workers=WORKERS, use_multiproce

## Adding some new features...
The following was done with:
`python src/data/tree_diffs.py  data/processed/competitions/ . --ignore_comments --python_only --git_context 3 --ignore_empty_lines --ignore_line_shuffle --n_workers 1 `

In [41]:
MORE_FEATURES_PATH = "/homes/gws/mikeam/RobustDataScience/diffs_test.jsonl"
!wc -l $MORE_FEATURES_PATH

224532 /homes/gws/mikeam/RobustDataScience/diffs_test.jsonl


Note how many more examples we got, I think by decreasing the context between lines such that we have smaller hunks.

In [42]:
more_features_examples = !shuf -n 100000 $MORE_FEATURES_PATH
more_features_examples = [json.loads(x) for x in more_features_examples]

In [43]:
for diff in more_features_examples[:40]:
    display_kaggle_diff(diff)

data/processed/competitions/zillow-prize-1/jaccojurg/30688082.json
----------------------------------------------------------------------------------------------------
 plt.figure(figsize=(15,10))
 
-Countries = ['Italy', 'Netherlands','Sweden','Germany', 'China','US','Belgium', 'Japan','France']
+Countries = ['Italy', 'Netherlands - Netherlands']
 #Countries = ['Italy', 'Netherlands', 'France'] #, 'Belgium'] #, 'Belgium','France']
 for country in Countries:
 
-#for r in range(len(info)):
-#    plt.annotate(info.loc[r]['event'],(info.loc[r]['dayssince'],info.loc[r]['CasesPerM']))
+for r in range(len(info)):
+    plt.annotate(info.loc[r]['event'],(info.loc[r]['dayssince'],info.loc[r]['CasesPerM']),
+                 xytext = (-10+info.loc[r]['dayssince'],info.loc[r]['CasesPerM']-1),
+                 #color=c[info.loc[r]['location']],
+                 arrowprops=dict(arrowstyle="->",connectionstyle="angle3"),
+                 horizontalalignment='left', verticalalignment='center')
 
 

In [18]:
def show_diffs_that_match_regex(diffs,regex,limit=100):
    matches = 0
    for diff in diffs:
        if re.search(regex,diff["cell_diff"]):
            display_kaggle_diff(diff)
            matches += 1
        if matches == limit:
            break

In [19]:
show_diffs_that_match_regex(more_features_examples, "ttest_ind", limit=100)

data/processed/competitions/santander-customer-transaction-prediction/aaronl87/16375570.json
----------------------------------------------------------------------------------------------------

 import pandas as pd
-from pandas.plotting import scatter_matrix
 import numpy as np
-
 from scipy.stats import ttest_ind, levene
 
-%matplotlib inline
-
 import warnings


data/processed/competitions/titanic/andyyang/1342951.json
----------------------------------------------------------------------------------------------------

-# Statitic Test
+# Statitic Test, variable is continuous, so we choose T-test
 # H0: People survived and not survived have same fare, mean(survive_fare)=mean(non_survive_fare)
+from scipy.stats import ttest_ind
 


data/processed/competitions/m5-forecasting-accuracy/armenabnousi/29611576.json
----------------------------------------------------------------------------------------------------
 import numpy as np # linear algebra
 import pandas as pd # data processing,

## Consolidated Diff Filters
The following were created using the consolidated diff filters:

In [46]:
CONSOLIDATED_DIFF_FILTERS_PATH = "/homes/gws/mikeam/RobustDataScience/data/processed/filtered_diffs.jsonl"
consolidated_examples = !shuf $CONSOLIDATED_DIFF_FILTERS_PATH
consolidated_examples = [json.loads(x) for x in consolidated_examples]

In [48]:
len(consolidated_examples)

93032

In [47]:
for diff in consolidated_examples[100:200]:
    display_kaggle_diff(diff)

data/processed/competitions/tmdb-box-office-prediction/takedown/13477503.json
----------------------------------------------------------------------------------------------------

-test['revenue'] =  np.expm1(test["xgbfinal"])
+test['revenue'] =  np.expm1(test["lgbfinal"])
 test[['id','revenue']].head()
data/processed/competitions/severstal-steel-defect-detection/watanabe2362/20644177.json
----------------------------------------------------------------------------------------------------
 def rle2mask(rle):
 def mask2rle(mask):
-    if np.sum(mask) == 0: return '1 1'
+    if np.sum(mask) == 0: return ''
     ar = mask.flatten(order='F')
data/processed/competitions/aerial-cactus-identification/kshashankrao/15278097.json
----------------------------------------------------------------------------------------------------
train_dataset = Dataset(data = X_train, transform=image_transform)
-test_dataset = Dataset(data = X_test, transform=image_transform)
+test_dataset = Dataset(data = X_val

In [59]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*ttest_ind", limit=100)

In [60]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*KMeans", limit=100)

data/processed/competitions/new-york-city-taxi-fare-prediction/yairhadad1/12395801.json
----------------------------------------------------------------------------------------------------
-cls_k_means = KMeans(n_clusters=2)
+cls_k_means = KMeans(n_clusters=3)
 cls_k_means.fit(k_mean_X)
data/processed/competitions/sf-crime/nguyenbaopc/22348067.json
----------------------------------------------------------------------------------------------------
-kmeans = KMeans(n_clusters=4)
+kmeans = KMeans(n_clusters=14)
 kmeans.fit(df_input)
data/processed/competitions/nyc-taxi-trip-duration/priyanka13/1441720.json
----------------------------------------------------------------------------------------------------
-kmeans = KMeans(n_clusters=2, random_state=0).fit(xkdt)
+kmeans = KMeans(n_clusters=3, random_state=0).fit(xkdt)
 test['dropoff_id']=kmeans.labels_
data/processed/competitions/bosch-production-line-performance/egdman/379461.json
---------------------------------------------------------

In [61]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*dropna", limit=100)

data/processed/competitions/house-prices-advanced-regression-techniques/plasticgrammer/4889718.json
----------------------------------------------------------------------------------------------------
-skewed = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
+skewed = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
 skewed_feats = skewed[skewed > 1].index
 print(skewed_feats)
data/processed/competitions/titanic/headsortails/1151431.json
----------------------------------------------------------------------------------------------------
-df = train.loc[:,tcols].dropna()
+df = training.loc[:,tcols].dropna()
 X = df.loc[:,cols]
data/processed/competitions/ga-customer-revenue-prediction/danofer/5722315.json
----------------------------------------------------------------------------------------------------
+    df.dropna(how="all",axis=1,inplace=True)
     df.drop([c for c in list_single_value if c in df.columns], axis=1, inpl

In [62]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*Dropout", limit=100)

data/processed/competitions/tweet-sentiment-extraction/lomen0857/36358298.json
----------------------------------------------------------------------------------------------------
-    x2 = tf.keras.layers.Dropout(0.13)(x[0]) 
+    x2 = tf.keras.layers.Dropout(0.15)(x[0]) 
     x2 = tf.keras.layers.Conv1D(1,1)(x2)
data/processed/competitions/jigsaw-unintended-bias-in-toxicity-classification/cevangelist/12729100.json
----------------------------------------------------------------------------------------------------
-    drop_0 = L.SpatialDropout1D(0.2)(emb)
+    drop_0 = L.Dropout(0.5, seed=42)(emb)
+    bi_lstm_0 = L.Bidirectional(L.CuDNNLSTM(RECURRENT_UNITS, return_sequences=False))(drop_0)
data/processed/competitions/Kannada-MNIST/mak4alex/21546456.json
----------------------------------------------------------------------------------------------------
-    con_drop_layer1 = Dropout(0.25)(relu_layer2)
-    max_pool_layer1 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(con_drop_layer

In [63]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*chisquare", limit=100)

In [64]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*LinearRegression", limit=100)

data/processed/competitions/house-prices-advanced-regression-techniques/y2kshehan/2007135.json
----------------------------------------------------------------------------------------------------
-lin_reg_pl = LinearRegression()
 #Predicting the SalePrice using cross validation (KFold method)
-y_pred_pl = cross_val_predict(lin_reg_pl, X_poly, y, cv=6 )
+y_pred_pl = cross_val_predict(lin_reg_pl, X_poly, y, cv=10 )
 #Polynominal Regression Accuracy with cross validation
 accuracy_pl = metrics.r2_score(y, y_pred_pl)
data/processed/competitions/restaurant-revenue-prediction/ani310/574131.json
----------------------------------------------------------------------------------------------------
-cls = linear_model.LinearRegression()
+cls = RandomForestRegressor()
 cls.fit(xTrain, yTrain)
data/processed/competitions/covid19-global-forecasting-week-3/letili0417/31396638.json
----------------------------------------------------------------------------------------------------
-        xmodel1 = l

In [65]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*wilcoxon", limit=100)

In [66]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*SGD", limit=100)

data/processed/competitions/online-sales/sgrsgrsgr/6969291.json
----------------------------------------------------------------------------------------------------
-clf = SGDClassifier(loss='log', penalty='none', max_iter=1000, fit_intercept=True, random_state=1234)
+clf = SGDClassifier(loss='log', penalty='L2', max_iter=5000, fit_intercept=True, random_state=1234)
 clf.fit(X_train, y_train)
data/processed/competitions/histopathologic-cancer-detection/artgor/7509653.json
----------------------------------------------------------------------------------------------------
-optimizer = optim.SGD(model_conv.parameters(), lr=0.1, momentum=0.9)
+optimizer = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)
 exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
data/processed/competitions/plant-seedlings-classification/allunia/18786594.json
----------------------------------------------------------------------------------------------------
-optimizer_ft = optim.S

In [67]:
show_diffs_that_match_regex(consolidated_examples, "^[\+-].*BERT", limit=100)

data/processed/competitions/tweet-sentiment-extraction/drhouse3/33634205.json
----------------------------------------------------------------------------------------------------
-model_config = transformers.AlbertConfig.from_pretrained(config.BERT_PATH)
+model_config = transformers.RobertaConfig.from_pretrained(config.BERT_PATH)
 model_config.output_hidden_states = True
data/processed/competitions/tweet-sentiment-extraction/viswajithkn/32086267.json
----------------------------------------------------------------------------------------------------
-    model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
+    model_config = transformers.RobertaConfig.from_pretrained(config.roberta_path)
     model = BertBaseQA(768, 2,model_config).to(device)
data/processed/competitions/jigsaw-multilingual-toxic-comment-classification/bamps53/30818481.json
----------------------------------------------------------------------------------------------------
-class BERTDatasetTest:
- 

## Wait, do we want to keep positional args afterall?


In [70]:
POSITIONAL_ARGS_PATH="~/RobustDataScience/data/processed/filtered_diffs_with_args.jsonl"
positional_args_examples = !shuf $POSITIONAL_ARGS_PATH
positional_args_examples = [json.loads(x) for x in positional_args_examples]

In [69]:
!shuf $POSITIONAL_ARGS_PATH

shuf: ./data/processed/filtered_diffs_with_args.jsonl: No such file or directory


In [71]:
show_diffs_that_match_regex(positional_args_examples, "^[\+-].*ttest_ind", limit=100)

## Using keywords from libraries
The following examples use libraries mined with `/homes/gws/mikeam/RobustDataScience/src/data/scrape_library_structures.py`:

In [84]:
USING_LIBRARY_ARGS = "/homes/gws/mikeam/RobustDataScience/data/processed/filtered_with_lib_structure.txt"
using_lib_examples = !shuf $USING_LIBRARY_ARGS
using_lib_examples = [json.loads(x) for x in using_lib_examples]

In [85]:
for diff in using_lib_examples[:40]:
    display_kaggle_diff(diff)

data/processed/competitions/microsoft-malware-prediction/adityaecdrid/8717145.json
----------------------------------------------------------------------------------------------------

-train['first_4'] = train['MachineIdentifier'].apply(lambda x: x[:4])
+train['first_4'] = train['MachineIdentifier'].apply(lambda x: x[:4]).astype('category')
 


-test['first_4'] = test['MachineIdentifier'].apply(lambda x: x[:4])
+test['first_4'] = test['MachineIdentifier'].apply(lambda x: x[:4]).astype('category')
 


data/processed/competitions/pubg-finish-placement-prediction/praneethvarmaalluri/8130448.json
----------------------------------------------------------------------------------------------------
 config = tf.contrib.learn.RunConfig(tf_random_seed=42)
 feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(x_train_squad_fpp_scaled)
-dnn_reg_squad_fpp = tf.contrib.learn.DNNRegressor(hidden_units=[2500,2500,2500,1500,1000],optimizer = tf.train.AdamOptimizer(), activation_fn 